### Preparar ambiente

In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

Instalar PyAthena

In [4]:
!{sys.executable} -m pip install PyAthena

### Conectar con pyathena

In [5]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [6]:
import pandas as pd
import awswrangler as wr
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')

In [7]:
import geopandas as gpd
import glob
import os

### Eligiendo pais y periodo

In [8]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [9]:
pais = 'BR' # 'PE' 'BO' 'PY' 'UY'

In [10]:
fecha_inicio = '09-01-2020'
fecha_final = '09-30-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)

### Generar tablas de usuarios según regiones

In [11]:
start_time_day = time.time()
for fecha in fechas:
    query_fecha = ""
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(f"Fecha: {codigo_fecha}")
    starttime=time.time()
    tabla_final_usuario_dia = f'historico_usuarios_adms_{pais}_{codigo_fecha}'
    starttime=time.time()                      
    query = (f'''CREATE TABLE {tabla_final_usuario_dia} 
             WITH (    
         external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/Registro_usuarios_regiones/{pais}/{tabla_final_usuario_dia}',
         format = 'PARQUET',  
         parquet_compression = 'SNAPPY') AS 
         SELECT filtro.*,
                t2.codmunic,
                t2.nommunic,
                t2.cd_geocmi
         FROM (SELECT * 
               FROM historico_usuarios 
               WHERE month={fecha.month} 
                  AND day={day} 
                  AND iso_country_code='{pais}') AS filtro LEFT JOIN 
                                                (SELECT  *                                                            
                                                  FROM historico_latlong_{pais}) AS t2  
            ON filtro.lat_final=t2.lat_final 
            AND filtro.long_final=t2.long_final
            AND filtro.iso_country_code=t2.iso_country_code
            ''')
    cursor.execute(query) 
    print((time.time()-starttime)/60)   

Fecha: 901
0.17563842137654623
Fecha: 902
0.22659559647242228
Fecha: 903
0.15848911205927532
Fecha: 904
0.17461387713750204
Fecha: 905
0.17380834023157757
Fecha: 906
0.22534419298171998
Fecha: 907
0.15806831121444703
Fecha: 908
0.24216421445210776
Fecha: 909
0.24191630283991497
Fecha: 910
0.4822375575701396
Fecha: 911
0.15737799008687336
Fecha: 912
0.2424865166346232
Fecha: 913
0.17519477208455403
Fecha: 914
0.2425594170888265
Fecha: 915
0.259336793422699
Fecha: 916
0.139532999197642
Fecha: 917
0.17469517787297567
Fecha: 918
0.24183833599090576
Fecha: 919
0.15750907659530639
Fecha: 920
0.20748315652211508
Fecha: 921
0.17330997784932453
Fecha: 922
0.17339885234832764
Fecha: 923
0.19119613965352375
Fecha: 924
0.15659406979878743
Fecha: 925
0.15841564734776814
Fecha: 926
0.15787098407745362
Fecha: 927
0.17578618129094442
Fecha: 928
0.15783746639887491
Fecha: 929
0.28410178820292153
Fecha: 930
0.2078903555870056


pais = 'BR' # 'PE' 'BO' 'PY' 'UY'
try:
    fecha_inicio = '03-01-2020'
    fecha_final = '06-21-2020'
    fechas = pd.date_range(fecha_inicio, fecha_final)
    for fecha in fechas:
        day = str(fecha.day)
        if len(day) == 1:
            day = f'0{day}'
        codigo_fecha = str(fecha.month) + day
        print(codigo_fecha)

        tabla_datos_pais = f"usuarios_historico_adms_{pais}_{codigo_fecha}_10p"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)

        tabla_datos_pais = f"usuarios_historico_adms_{pais}_{codigo_fecha}_4p"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)
        
except:
    print('Tablas eliminadas')